In [25]:
!pip install lxml

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
import pandas as pd
import pickle
from bs4 import BeautifulSoup
import csv
from time import sleep
import requests
import re
import lxml
import string

translator = str.maketrans('', '', string.punctuation)

In [3]:
genre_tuples = [
    ('folk', 30, 'https://www.albumoftheyear.org/genre/5-folk/all/pitchfork/'), 
    ('indie_folk', 11, 'https://www.albumoftheyear.org/genre/16-indie-folk/all/'),
    ('folk_rock', 7, 'https://www.albumoftheyear.org/genre/14-folk-rock/all/pitchfork/'),
    ('alt_country', 8, 'https://www.albumoftheyear.org/genre/17-alt-country/all/'),
    ('hip_hip', 48, 'https://www.albumoftheyear.org/genre/3-hip-hop/all/pitchfork/'),
    ('pop', 15, 'https://www.albumoftheyear.org/genre/15-pop/all/pitchfork/'),
    ('alt_rock', 26, 'https://www.albumoftheyear.org/genre/2-alternative-rock/all/pitchfork/'),
    ('Rock', 120, 'https://www.albumoftheyear.org/genre/7-rock/all/pitchfork/'),
    ('experimental_rock', 8, 'https://www.albumoftheyear.org/genre/8-experimental-rock/all/pitchfork/')
]



def album_getter(tuples, album_id=800):
    
    for genre_tuple in tuples:
        name, pages, url  = genre_tuple
        
        albums = {}
        
        for page in range(1, pages):
            res = requests.get(url + str(page) + '/')
            print(name, page, res.status_code)
            soup = BeautifulSoup(res.content, 'lxml')
            for x in soup.find_all('div', {'class', 'albumListRow'}):
                
                score        = (x.find('div', {'class', 'scoreValue'}).text)
                album        = ((x.find('a').text.split('-')[1]).strip())
                artist       = ((x.find('a').text.split('-')[0]).strip())
                release_date = x.find('div', {'class', 'albumListDate'}).text

                albums[album_id] = {
                    'album_id'    : album_id,
                    'score'       : score,
                    'album'       : album,
                    'artist'      : artist,
                    'release_date': release_date
                }

                album_id = album_id + 1
            sleep(1)
            
        pickle.dump(albums, open(name + '_album_dict.pkl', 'wb'))

In [16]:
def lyric_getter():
    
    bad_urls = []
    
    list_of_genre_names = ['alt_country', 'hip_hip', 'pop', 'alt_rock', 'Rock', 'experimental_rock']
    
    for name in list_of_genre_names:
    
        with open(name + '_album_dict.pkl', 'rb') as f:
            album_dict = pickle.load(f)

        song_id = 0

        base_url = 'https://genius.com/albums/'

        song_dictionary = {}

        for album_id in album_dict:
            try:

                album         = album_dict[album_id]['album']
                artist        = album_dict[album_id]['artist']

                url_artist = artist.translate(translator).replace(' ', '-')
                url_album  = album.translate(translator).replace(' ', '-')

                album_href = base_url + url_artist + '/' + url_album + '/'

                album_result = requests.get(album_href)

                sleep(1)

                if album_result.status_code == 200:

                    album_soup = BeautifulSoup(album_result.content, 'lxml')

                    track_counts  = album_soup.find_all('div', {'class': 'chart_row-number_container chart_row-number_container--align_left'})
                    track_listing = album_soup.find_all('div', {'class': 'chart_row-content'})

                    for count, song in enumerate(track_listing):

                        track_count = track_counts[count].find('span').text.replace('\n', '')
                        song_title  = track_listing[count].find('h3').text.strip().split('\n')[0]
                        href        = track_listing[count].find('a')['href']

                        song_results = requests.get(href)
                        song_soup    = BeautifulSoup(song_results.content, 'lxml')
                        lyrics       = re.sub('(\\n){2}', '\\n', song_soup.find('p').text).replace('\n', ' n ')

                        song_info = [
                            str(song_id),
                            str(album_id),
                            artist,
                            album,
                            album_href,
                            song_title,
                            track_count,
                            lyrics,
                            href,
                        ]

                        with open(name + 'song_lyrics.psv', 'a+') as f:
                            f.write('|'.join(song_info) + '\n')

                        song_dictionary[song_id] = {
                            'song_id'      : song_id,
                            'album_id'     : album_id,
                            'artist'       : artist,
                            'album'        : album,
                            'album_href'   : album_href,
                            'song_title'   : song_title,
                            'track_count'  : track_count,
                            'lyrics'       : lyrics,
                            'href'         : href
                        }

                        song_id = song_id + 1

                        sleep(1)

                else:
                    bad_urls.append(album_id)
                    pass
            except:
                bad_urls.append(album_id)
                pass
        
    pickle.dump(song_dictionary, open(name + 'song_dictionary.pkl', 'wb'))

    pickle.dump(bad_urls, open('bad_urls.pkl', 'wb'))

In [17]:
album_getter(genre_tuples)

In [18]:
lyric_getter()